In [1]:
from tool import *

In [2]:
with open('D:/Data/ODM_SR.db', 'rb') as f:
    sr = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    site = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)

In [3]:
thisweek = get_weekname(datetime.date.today())

In [4]:
sr['Week Name'] = sr['Ship Date'].map(get_weekname)

In [5]:
sr = sr.reset_index().merge(site[['To Site', 'Region', 'Country', 'Subsidiary']])

In [6]:
sr.loc[sr['Division']=='PC', 'SCM'] = sr.loc[sr['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sr.loc[sr['Division']=='PC', 'MKT'] = sr.loc[sr['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sr.loc[sr['Division']=='MNT', 'SCM'] = sr.loc[sr['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sr.loc[sr['Division']=='MNT', 'MKT'] = sr.loc[sr['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
us_b2b = sr.loc[sr['To Site'].str.contains('ENUS') & (sr['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].min())
sr.loc[sr['To Site'].str.contains('ENUS') & (sr['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'MKT'] = us_b2b

In [7]:
sr = sr[sr['Week Name']==get_weekname_from(thisweek, -1)].fillna('N/A').pivot_table('Ship', index=['From Site', 'BL No', 'Region', 'Country', 'Subsidiary', 'To Site', 'Mapping Model.Suffix', 'PO No.', 'PO1 No', 'SO No.', 'Final Destination', 'Ship Date', 'SCM', 'MKT'], aggfunc=sum).reset_index()
# sr = sr[sr['Week Name'].isin([get_weekname_from(thisweek, 0)])].fillna('N/A').pivot_table('Ship', index=['From Site', 'BL No', 'Region', 'Country', 'Subsidiary', 'To Site', 'Mapping Model.Suffix', 'PO No.', 'PO1 No', 'SO No.', 'Final Destination', 'Ship Date', 'SCM', 'MKT'], aggfunc=sum).reset_index()

In [8]:
sr = sr[sr['From Site'].str.contains('TCL')]

# 매출 담당자
### 1) EU : 전유미 선임
### 2) MEA : 오설 책임
### 3) SEA : 현은지 선임
### 4) NA : 김정미 사원
### 5) SCA : 최경남 책임
### 6) Korea : 김준현 사원, 김윤정 선임, 김승현 기정

In [9]:
sr

,From Site,BL No,Region,Country,Subsidiary,To Site,Mapping Model.Suffix,PO No.,PO1 No,SO No.,Final Destination,Ship Date,SCM,MKT,Ship
64,OS_TCL_CN_P,PLISZ4E14803,SCA,Columbia,LGECB,ESCB,22MR410-B.AWPQ,JGMZ23034335,1024557,380172200,COBUN,2023-12-28,장재민,손승아,840.0
65,OS_TCL_CN_P,PLISZ4E14885,SEA,Vietnam,LGEVH,EAVH_HCM,27MR400-B.ATVQ,JGMZ23036583,296680,380173096,VNSGN,2023-12-25,김송이,박진아,512.0
66,OS_TCL_CN_P,PLISZ4E14922,MEA,Turkey,LGETK,EMTK,22MR410-B.APDQ,JGMZ23034344,1378790,380172214,TRTUZ,2023-12-30,오석현,상민철,1680.0
67,OS_TCL_CN_P,PLISZ4E14922,MEA,Turkey,LGETK,EMTK,24MR400-B.APDQ,JGMZ23034345,1378790,380172214,TRTUZ,2023-12-30,오석현,상민철,800.0
68,OS_TCL_CN_P,PLISZ4E14922,MEA,Turkey,LGETK,EMTK,27MR400-B.APDQ,JGMZ23034346,1378790,380172214,TRTUZ,2023-12-30,오석현,상민철,512.0
69,OS_TCL_CN_P,PLISZ4E14925,SEA,Philippines,LGEPH,EACM,22MR410-B.APHQ,JGMZ23034300,1021025,380172179,PHMNL,2023-12-26,김송이,이기현,840.0
70,OS_TCL_CN_P,PLISZ4E14925,SEA,Philippines,LGEPH,EACM,24MR400-B.APHQ,JGMZ23034301,1021025,380172179,PHMNL,2023-12-26,김송이,이기현,1920.0
71,OS_TCL_CN_P,PLISZ4E14926,EU,France,LGEFS,EEFS,27MR400-B.AEUQ,JGMZ23034306,1176864,380172184,FRSRV,2023-12-28,송명규,오세림,512.0
72,OS_TCL_CN_P,PLISZ4E14927,INDIA,India,LGEIL,EAIL,22MR410-B.ATRQ,JGMZ23034439,975599,380172253,INNSA,2023-12-28,오석현,고주현,840.0
73,OS_TCL_CN_P,PLISZ4E14927,INDIA,India,LGEIL,EAIL,24MR400-B.ATRQ,JGMZ23034440,975599,380172253,INNSA,2023-12-28,오석현,고주현,800.0


In [10]:
bl_num = ['PLISZ4E14922']
c1 = sr['BL No'].isin(bl_num)

In [11]:
sr[c1].to_clipboard(index=False)
sr[c1]

,From Site,BL No,Region,Country,Subsidiary,To Site,Mapping Model.Suffix,PO No.,PO1 No,SO No.,Final Destination,Ship Date,SCM,MKT,Ship
66,OS_TCL_CN_P,PLISZ4E14922,MEA,Turkey,LGETK,EMTK,22MR410-B.APDQ,JGMZ23034344,1378790,380172214,TRTUZ,2023-12-30,오석현,상민철,1680.0
67,OS_TCL_CN_P,PLISZ4E14922,MEA,Turkey,LGETK,EMTK,24MR400-B.APDQ,JGMZ23034345,1378790,380172214,TRTUZ,2023-12-30,오석현,상민철,800.0
68,OS_TCL_CN_P,PLISZ4E14922,MEA,Turkey,LGETK,EMTK,27MR400-B.APDQ,JGMZ23034346,1378790,380172214,TRTUZ,2023-12-30,오석현,상민철,512.0


In [12]:
# bl_num = sr.loc[i]['BL No']
region = sr[c1]['Region'].unique()[0]
country = sr[c1]['Country'].unique()[0]
weekname = sr[c1]['Ship Date'].map(get_weekname).unique()[0]

email_subject = f'제목 : 중국 ODM社 TCL 모니터 제품 중계무역 {region}향({country}) 매출화 요청 {bl_num}'
email_body = f'''안녕하십니까?

아래의 건으로 {weekname}주차 {region} 지역인 {country}향 중계무역 매출화 요청 드립니다.
상세한 내용은 첨부의 B/L 참조하시기 바랍니다.
'''
email_subject_kor = f'제목 : 중국 ODM社 TCL 모니터 제품 한국향 통관 요청 {bl_num}'
email_body_kor = f'''
안녕하십니까?

수신 : Pantos 김준현 사원님
아래의 건으로 중국 모니터 업체인 TCL사로부터 {weekname}주차 한국으로 선적된 모니터 제품 Port에 도착되면 통관 진행 요청 드립니다.
상세한 내용은 첨부의 선적 서류 참조하시기 바랍니다.
'''    

In [13]:
if sr[c1]['Country'].values[0] == 'Korea':
    print(email_subject_kor)
    print(email_body_kor)
elif region == 'SEA':
    print(email_subject)
    print(email_body)
    nw = pd.read_excel(get_filename(), skiprows=1)['Net Wt'].dropna().values[-1]
    print(f'Net Weight: {nw} KG')
else:
    print(email_subject)
    print(email_body)

제목 : 중국 ODM社 TCL 모니터 제품 중계무역 MEA향(Turkey) 매출화 요청 ['PLISZ4E14922']
안녕하십니까?

아래의 건으로 2023-12-25(W52)주차 MEA 지역인 Turkey향 중계무역 매출화 요청 드립니다.
상세한 내용은 첨부의 B/L 참조하시기 바랍니다.

